<a href="https://colab.research.google.com/github/CKristensen/OsloCityBikes_Analysis/blob/master/machine_learning/Bysykkel_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import psycopg2
import requests
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import time
import io
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Load data from SQL-query directly from db:

In [ ]:
# Database variabler
PASS = "password"
USER_NAME = "username"
HOST = "host"
DATABASE = 'database'

# Create engine:
engine = create_engine(f'postgresql+psycopg2://{USER_NAME}:{PASS}@{HOST}/{DATABASE}')

# SQL query:
sql= '''select s.station_key,s.military_hour,s.trips_started,s.trips_ended, s.trips_ended+s.trips_started as all_trips,
s.air_temperatur_celsius, s.precipitation_mm, s.wind_speed_ms, s.date_actual, s.day_of_week, s.day_name, 
s.month_actual, s.day_of_month, s.quarter_actual, s.day_of_year, s.is_holiday, s.is_strike 
from star.stationtripslast s
where date_actual>='2019-04-01';'''

# Read in data from query to df
df_features= pd.read_sql_query(sql, engine, index_col=None)

# View all columns!
pd.set_option('display.max_columns', None)

# Make station_key numeric for easier merge later
df_features['station_key']=pd.to_numeric(df_features['station_key'])

print(df_features.head(5))
print(df_features.info())

   station_key  military_hour  trips_started  trips_ended  all_trips  \
0          509              1              0            0          0   
1          510              1              0            0          0   
2          511              1              0            0          0   
3          512              1              0            0          0   
4          513              1              0            0          0   

   air_temperatur_celsius  precipitation_mm  wind_speed_ms date_actual  \
0                     1.6               0.0            1.8  2019-04-02   
1                     1.6               0.0            1.8  2019-04-02   
2                     1.6               0.0            1.8  2019-04-02   
3                     1.6               0.0            1.8  2019-04-02   
4                     1.6               0.0            1.8  2019-04-02   

   day_of_week   day_name  month_actual  day_of_month  quarter_actual  \
0            2  Tuesday               4          

In [ ]:
# Import csv with updated elevation column.

#from google.colab import files
#upload_csv = files.upload()

ele=pd.read_csv('/content/elevation.csv')
ele=ele.rename(columns={'new':'station_key'})
ele.head(2)

,station_key,station_name,description,latitude_y,longitude_y,elevation_x
0,769,NaN,NaN,59.915553,10.751323,22.833595
1,767,NaN,NaN,59.918217,10.756065,16.531841


In [ ]:
ele.sort_values(by='station_key', ascending=False)

,station_key,station_name,description,latitude_y,longitude_y,elevation_x
313,1919,Kværnerveien,Ved Kværnerveien 5,59.905911,10.778592,28.570000
312,1755,Aker Brygge,ved trikkestopp,59.911184,10.730035,8.640000
311,1101,Stortingstunellen,langs Rådhusgata,59.910653,10.737365,22.854000
310,1023,Professor Aschehougs plass,ved Kristian IVs gate,59.914767,10.740971,32.050000
309,1009,Borgenveien,ved Holmenveien,59.942742,10.703833,95.000000
...,...,...,...,...,...,...
156,381,Grønlands torg,ved Tøyenbekken,59.912520,10.762240,12.856841
159,380,Bentsebrugata,rett over busstoppet,59.939230,10.759170,83.595863
138,379,NaN,NaN,59.916798,10.758138,17.349005
48,378,Colosseum Kino,langs Fridtjof Nansens vei,59.929853,10.711515,57.960068


Merge together with left join. 
Describing the data:

In [ ]:
df_merge= pd.merge(df_features,ele[['station_key','elevation_x']], on='station_key',how='left')

# Turning elevation back to string:
df_features['station_key']=df_features['station_key'].astype(str)

df_merge.head(5)


,station_key,military_hour,trips_started,trips_ended,all_trips,air_temperatur_celsius,precipitation_mm,wind_speed_ms,date_actual,day_of_week,day_name,month_actual,day_of_month,quarter_actual,day_of_year,is_holiday,is_strike,elevation_x
0,509,1,0,0,0,1.6,0.0,1.8,2019-04-02,2,Tuesday,4,2,2,92,0,0,26.864119
1,510,1,0,0,0,1.6,0.0,1.8,2019-04-02,2,Tuesday,4,2,2,92,0,0,89.752731
2,511,1,0,0,0,1.6,0.0,1.8,2019-04-02,2,Tuesday,4,2,2,92,0,0,36.407383
3,512,1,0,0,0,1.6,0.0,1.8,2019-04-02,2,Tuesday,4,2,2,92,0,0,62.905735
4,513,1,0,0,0,1.6,0.0,1.8,2019-04-02,2,Tuesday,4,2,2,92,0,0,102.912750


In [ ]:
df_merge.info()
df_merge.describe()


# Checking first and last date:
print(df_merge['date_actual'].min())
print(df_merge['date_actual'].max())

# Checking for null-values:
df_merge.isna().sum()

# Filling null-values by sea-level at 10m
df_merge['elevation_x']=df_merge['elevation_x'].fillna(10)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 3485664 entries, 0 to 3485663
Data columns (total 18 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   station_key             int64  
 1   military_hour           int64  
 2   trips_started           int64  
 3   trips_ended             int64  
 4   all_trips               int64  
 5   air_temperatur_celsius  float64
 6   precipitation_mm        float64
 7   wind_speed_ms           float64
 8   date_actual             object 
 9   day_of_week             int64  
 10  day_name                object 
 11  month_actual            int64  
 12  day_of_month            int64  
 13  quarter_actual          int64  
 14  day_of_year             int64  
 15  is_holiday              int64  
 16  is_strike               int64  
 17  elevation_x             float64
dtypes: float64(4), int64(12), object(2)
memory usage: 505.3+ MB
2019-04-02
2020-09-30


Dataprep:
- Histogram of features
- Scale numeric columns
- Embedding of station-name
- OneHot Vector of days(?), months, quarter and hour
- Make 

In [ ]:
# Legger til kolonne om det er helg eller ikke
df_merge['is_weekend']= np.where(df_features['day_name'].isin(['Sunday','Saturday']),1,0)

df_merge['is_weekend'].head(5)

0    0
1    0
2    0
3    0
4    0
Name: is_weekend, dtype: int64

In [ ]:
# OHE av day_name (not used yet):

days_encoder= OneHotEncoder(sparse=False)

days= df_f[['day_name']]
print(days.head(10))

days_ohe= pd.DataFrame(days_encoder.fit_transform(days))
days_ohe.columns = days_encoder.get_feature_names(['day_name'])
print(days_ohe.head(5))

In [ ]:
# Creating final df:
df_merge.columns

df_all= df_merge[['station_key', 'military_hour',
       'all_trips', 'air_temperatur_celsius', 'precipitation_mm',
       'wind_speed_ms', 'elevation_x', 'date_actual', 'day_of_week',
       'month_actual', 'day_of_month', 'quarter_actual', 'day_of_year',
       'is_holiday', 'is_weekend']]

df_all.head(5)

,station_key,military_hour,all_trips,air_temperatur_celsius,precipitation_mm,wind_speed_ms,elevation_x,date_actual,day_of_week,month_actual,day_of_month,quarter_actual,day_of_year,is_holiday,is_weekend
0,509,1,0,1.6,0.0,1.8,26.864119,2019-04-02,2,4,2,2,92,0,0
1,510,1,0,1.6,0.0,1.8,89.752731,2019-04-02,2,4,2,2,92,0,0
2,511,1,0,1.6,0.0,1.8,36.407383,2019-04-02,2,4,2,2,92,0,0
3,512,1,0,1.6,0.0,1.8,62.905735,2019-04-02,2,4,2,2,92,0,0
4,513,1,0,1.6,0.0,1.8,102.912750,2019-04-02,2,4,2,2,92,0,0


Setting date as index - and splitting into test & training:

In [ ]:
# Set index to date and reset index:
df_all= df_all.set_index('date_actual').sort_index()
print(df_all.head(10))

# Train, test, split:
split_date = pd.to_datetime('2020-08-01').date()
train= df_all.loc[df_all.index <= split_date].copy()
test= df_all.loc[df_all.index > split_date].copy()

#Sjekker at splitten er riktig
print(test.head(5))
print(train.head(6))

# Split into x and y:
y_train= train['all_trips']
x_train = train.loc[:,train.columns!='all_trips']

y_test= test['all_trips']
x_test = test.loc[:,test.columns!='all_trips']


             station_key  military_hour  all_trips  air_temperatur_celsius  \
date_actual                                                                  
2019-04-02           509              1          0                     1.6   
2019-04-02           514             17          0                     7.9   
2019-04-02           513             17          0                     7.9   
2019-04-02           512             17          0                     7.9   
2019-04-02           511             17          0                     7.9   
2019-04-02           510             17          0                     7.9   
2019-04-02           509             17          0                     7.9   
2019-04-02           508             17          0                     7.9   
2019-04-02           507             17          0                     7.9   
2019-04-02           506             17          0                     7.9   

             precipitation_mm  wind_speed_ms  elevation_x  day_

In [ ]:
train.head()

,station_key,military_hour,all_trips,air_temperatur_celsius,precipitation_mm,wind_speed_ms,elevation_x,day_of_week,month_actual,day_of_month,quarter_actual,day_of_year,is_holiday,is_weekend,is_strike
date_actual,,,,,,,,,,,,,,,
2019-04-02,1009,0,0,1.25,0.0,2.157143,95.000,2,4,2,2,92,0,0,0
2019-04-02,1023,0,0,1.25,0.0,2.157143,32.050,2,4,2,2,92,0,0,0
2019-04-02,1101,0,0,1.25,0.0,2.157143,22.854,2,4,2,2,92,0,0,0
2019-04-02,1755,0,0,1.25,0.0,2.157143,8.640,2,4,2,2,92,0,0,0
2019-04-02,1919,0,0,1.25,0.0,2.157143,28.570,2,4,2,2,92,0,0,0


Naive baseline model:
- Using mean

In [ ]:
y_test_naive= np.mean(y_test)*np.ones(y_test.shape)

RMSE_naive= np.sqrt(mean_squared_error(y_test, y_test_naive))
print(RMSE_naive)

4.728056506184842


XGBoost model:

In [ ]:
model = xgb.XGBRegressor(max_depth=10, n_estimators=1100, eta=0.2, seed=42)

model.fit(x_train, y_train,
          eval_metric='rmse',
          eval_set=[(x_train, y_train), (x_test, y_test)],
          early_stopping_rounds=10,
          verbose=True)

# validation_0-rmse:2.45602	validation_1-rmse:3.31787

[16:49:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:4.16831	validation_1-rmse:4.93039
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:4.01313	validation_1-rmse:4.72065
[2]	validation_0-rmse:3.87715	validation_1-rmse:4.53454
[3]	validation_0-rmse:3.76863	validation_1-rmse:4.37439
[4]	validation_0-rmse:3.66518	validation_1-rmse:4.23317
[5]	validation_0-rmse:3.57625	validation_1-rmse:4.11339
[6]	validation_0-rmse:3.50158	validation_1-rmse:4.01492
[7]	validation_0-rmse:3.42268	validation_1-rmse:3.91264
[8]	validation_0-rmse:3.35748	validation_1-rmse:3.83028
[9]	validation_0-rmse:3.30436	validation_1-rmse:3.76955
[10]	validation_0-rmse:3.24564	validation_1-rmse:3.70239
[11]	validation_0-rmse:3.19758	validation_1-rmse:3.65011
[12]	validation_0-rmse:3.1549	validation

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.2, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=1, verbosity=1)

In [ ]:
model2 = xgb.XGBRegressor(max_depth=12, n_estimators=1100, eta=0.2, seed=42)

model2.fit(x_train, y_train,
          eval_metric='rmse',
          eval_set=[(x_train, y_train), (x_test, y_test)],
          early_stopping_rounds=10,
          verbose=True)

# validation_0-rmse:2.65314	validation_1-rmse:3.28479

[16:58:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:4.15078	validation_1-rmse:4.89913
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:3.97131	validation_1-rmse:4.64768
[2]	validation_0-rmse:3.82438	validation_1-rmse:4.43107
[3]	validation_0-rmse:3.6884	validation_1-rmse:4.24368
[4]	validation_0-rmse:3.56375	validation_1-rmse:4.07904
[5]	validation_0-rmse:3.4646	validation_1-rmse:3.94624
[6]	validation_0-rmse:3.37246	validation_1-rmse:3.83687
[7]	validation_0-rmse:3.28689	validation_1-rmse:3.7312
[8]	validation_0-rmse:3.21927	validation_1-rmse:3.65801
[9]	validation_0-rmse:3.15307	validation_1-rmse:3.58473
[10]	validation_0-rmse:3.08743	validation_1-rmse:3.51498
[11]	validation_0-rmse:3.03389	validation_1-rmse:3.46308
[12]	validation_0-rmse:2.9826	validation_1-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.2, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=12, min_child_weight=1, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=1, verbosity=1)

In [ ]:
model3 = xgb.XGBRegressor(max_depth=20, n_estimators=1100, eta=0.2, seed=42)

model3.fit(x_train, y_train,
          eval_metric='rmse',
          eval_set=[(x_train, y_train), (x_test, y_test)],
          early_stopping_rounds=10,
          verbose=True)

[17:04:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:4.05329	validation_1-rmse:4.76932
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:3.79614	validation_1-rmse:4.42865
[2]	validation_0-rmse:3.56047	validation_1-rmse:4.12136
[3]	validation_0-rmse:3.35955	validation_1-rmse:3.87969
[4]	validation_0-rmse:3.17007	validation_1-rmse:3.67271
[5]	validation_0-rmse:3.00387	validation_1-rmse:3.51413
[6]	validation_0-rmse:2.86575	validation_1-rmse:3.39883
[7]	validation_0-rmse:2.7347	validation_1-rmse:3.3084
[8]	validation_0-rmse:2.61647	validation_1-rmse:3.24191
[9]	validation_0-rmse:2.50944	validation_1-rmse:3.19438
[10]	validation_0-rmse:2.41385	validation_1-rmse:3.1623
[11]	validation_0-rmse:2.32605	validation_1-rmse:3.14202
[12]	validation_0-rmse:2.24163	validation_1

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.2, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=1100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=1, verbosity=1)

Importance of features plot:

In [ ]:
# Importance of features:
# Can see that is_holiday is not very important, same with month actual. Probably because it can be inferred from day of year, 
# which is important

_ = plot_importance(model, height=0.9)

# får prediksjoner:
X_test_pred = model.predict(x_test)

Plotting performance of model

In [ ]:
# All data vs predicted data for test:
plt.figure(figsize=(15,3))
plt.title("Prediction vs actual")
plt.xlabel("Time")
plt.ylabel("Count of biketrips")
plt.plot(df_all.index, df_all['all_trips'])
plt.plot(x_test.index, X_test_pred, label="prediction")


plt.figure(figsize=(15,3))
plt.title("Prediction vs test")
plt.xlabel("Time")
plt.ylabel("Count of biketrips")
plt.plot(x_test.index, y_test, label='test')
plt.plot(x_test.index, X_test_pred, label="prediction")

Plotting test vs predictions:

In [ ]:
plt.figure(figsize=(15,3))
plt.title("Prediction vs test")
plt.xlabel("Time")
plt.ylabel("Count of biketrips")
plt.plot(x_test.index, y_test, label='test')
plt.plot(x_test.index, X_test_pred, label="prediction")